# Team with most points across season to date


### Assumptions made
- Based on the idea that the players with the CURRENT highest points, will continue to produce the greatest no. of points
- Players with <90% of playing have been removed
- INCORRECT - Currently doesn't account for squad rules
- Assumes budget of £100 million


### Selection Procces Method
- Initially picks the players with highest points from the following positions
-- 2 GKs
-- 5 DEFs
-- 5 MIDs
-- 3 ATTs

- Then (assuming the team will value > £100m) the picked player with the lowest delta will be removed
- The next_highest player from that position will then fill this place.
- These previous two steps will then iterate until the squad value is <= £100m



In [5]:
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [6]:
# Make a get request to api
response = requests.get("https://fantasy.premierleague.com/drf/bootstrap-static")
data = response.json()

# Print the status code of the response.
#print(response.content)


In [127]:
#Extract json to pandas dataframe
elements = data['elements']
ds = pd.DataFrame(elements)

#Converts cost into millions
ds['now_cost'] = ds['now_cost'] /10

#Removes any player with NaN chance of playing
ds = ds.dropna(how='any', subset=['chance_of_playing_next_round'])

#Removes any player with <90% chance of playing
is_likelyToPlay = ds['chance_of_playing_next_round']>90
ds = ds[is_likelyToPlay]

#Creation of positional datasets

is_gk = ds['element_type']==1
is_def = ds['element_type']==2
is_mid = ds['element_type']==3
is_att = ds['element_type']==4

gks = ds[is_gk]
defs = ds[is_def]
mids = ds[is_mid]
atts = ds[is_att]

In [128]:
# Gks add delta
z = np.polyfit(gks['now_cost'], gks['total_points'], 1)
p = np.poly1d(z)
gks['delta'] = gks['total_points'] - p(gks['now_cost'])
gks = gks.sort_values(by=['total_points'], ascending=False)
gks['pos'] = 'gk'

# Def add delta
z = np.polyfit(defs['now_cost'], defs['total_points'], 1)
p = np.poly1d(z)
defs['delta'] = defs['total_points'] - p(defs['now_cost'])
defs = defs.sort_values(by=['total_points'], ascending=False)
defs['pos'] = 'def'

# Mid add delta
z = np.polyfit(mids['now_cost'], mids['total_points'], 1)
p = np.poly1d(z)
mids['delta'] = mids['total_points'] - p(mids['now_cost'])
mids = mids.sort_values(by=['total_points'], ascending=False)
mids['pos'] = 'mid'

# Att Summary
z = np.polyfit(atts['now_cost'], atts['total_points'], 1)
p = np.poly1d(z)
atts['delta'] = atts['total_points'] - p(atts['now_cost'])
atts = atts.sort_values(by=['total_points'], ascending=False)
atts['pos'] = 'att'


C:\Users\domin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\domin\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\domin\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [129]:
#Dictionary to store the player with the next highest points by position
next_highest = {}
next_highest['gk'] = 3
next_highest['def'] = 6
next_highest['mid'] = 6
next_highest['att'] = 4

In [140]:
#Dictionary to store the player with the next highest points by position
next_highest = {}
next_highest['gk'] = 3
next_highest['def'] = 6
next_highest['mid'] = 6
next_highest['att'] = 4

squad = pd.DataFrame()
squad = pd.concat([squad, gks.head(2)])
squad = pd.concat([squad, defs.head(5)])
squad = pd.concat([squad, mids.head(5)])
squad = pd.concat([squad, atts.head(3)])

squad_cost = sum(squad['now_cost'])
print(squad[['second_name','total_points','delta','now_cost', 'pos']])
print('\n')
print('Initial Squad Cost: ' + str(squad_cost))

iteration = 1

while (squad_cost > 100):
    print('\n\nIteration ' + str(iteration))
    #Removes player with lowest delta
    cut_player = squad.sort_values(by=['delta'], ascending=True).head(1)
    squad = squad.sort_values(by=['delta'], ascending=True).iloc[1:]
    print('Cut player: ' + str(cut_player[['second_name']].iloc[0]['second_name']))
    
    if (cut_player[['pos']].iloc[0]['pos'] == 'gk'):
        replacement_player = gks.iloc[[next_highest['gk']]]
        next_highest['gk'] += 1
        
    elif (cut_player[['pos']].iloc[0]['pos'] == 'def'):
        replacement_player = defs.iloc[[next_highest['def']]]
        next_highest['def'] += 1
        
    elif (cut_player[['pos']].iloc[0]['pos'] == 'mid'):
        replacement_player = mids.iloc[[next_highest['mid']]]
        next_highest['mid'] += 1

    elif (cut_player[['pos']].iloc[0]['pos'] == 'att'):
        replacement_player = atts.iloc[[next_highest['att']]]
        next_highest['att'] += 1
        
        
    print('Replacement player: ' + str(replacement_player.iloc[0]['second_name']))
    squad = pd.concat([squad, replacement_player.head(1)])
    
    
    squad_cost = sum(squad['now_cost'])
    print('Updated Squad Cost: ' + str(squad_cost))
    iteration += 1
    
print('\nFINAL SQUAD')
squad = squad.sort_values(by=['pos'], ascending=True)
print(squad[['second_name','total_points','delta','now_cost', 'pos','team']])
print('\n')
squad_cost = sum(squad['now_cost'])
print('Final Squad Cost:\n' + str(squad_cost))

              second_name  total_points      delta  now_cost  pos
453                Lloris            80  34.333333       5.4   gk
393              Dubravka            79  41.600000       5.0   gk
323             Robertson           124  50.228634       6.7  def
322              van Dijk           113  45.679808       6.4  def
146  Luiz Moreira Marinho           105  48.431765       5.9  def
148               Rüdiger            92  33.281374       6.0  def
550               Doherty            90  48.484505       5.2  def
329                 Salah           162   2.819707      13.4  mid
152                Hazard           144  18.733456      11.1  mid
350              Sterling           133   4.784435      11.3  mid
45                 Fraser           111  59.458997       6.1  mid
327                  Mané           108   6.325629       9.5  mid
21             Aubameyang           134  24.594427      11.3  att
333               Firmino           104  21.030462       9.3  att
360       